In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import numpy as np


In [3]:
data=pd.read_csv('../2~3.데이터 수집 및 전처리/머신러닝 전처리 데이터.CSV',encoding='EUC-KR')

In [4]:
data.head(5)

,시군구,단지명,전용면적(㎡),계약년월,계약일,계약날짜,층,건축나이,거래건수,기준금리,법정동코드,지하철역수,거래금액(만원)
0,광주광역시 광산구 도산동,대덕1,59.3100,202303,18,2023-03-18,4,33,20,3.5,2920010200,1,9800
1,광주광역시 광산구 도산동,대주피오레1차,84.9800,202302,7,2023-02-07,2,23,14,3.5,2920010200,1,21000
2,광주광역시 광산구 도산동,대주피오레1차,84.9800,202305,31,2023-05-31,5,23,16,3.5,2920010200,1,22350
3,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,9,2023-05-09,5,22,16,3.5,2920010200,1,22400
4,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,13,2023-05-13,13,22,16,3.5,2920010200,1,22000


In [4]:
X = data[['전용면적(㎡)', '계약년월','층', '건축나이','거래건수','기준금리','법정동코드','지하철역수']]
y = data['거래금액(만원)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##### lrmodel = LinearRegression()
ridge = Ridge()
lasso = Lasso()
enet = ElasticNet()
d_tree = DecisionTreeRegressor()
rf = RandomForestRegressor()
model_xgb = XGBRegressor()
model_lgb = LGBMRegressor()

In [6]:



# 토큰화 방식 : 공백, 글자, 형태소 분석 으로 토큰화 한다.
from konlpy.tag import Okt

okt = Okt()
tokenized_data = []
data['단지명'] = data['단지명'].astype(str)
for sentence in data['단지명']:
    tokens = okt.morphs(sentence)
    tokenized_data.append(' '.join(tokens))
data['단지명'] = tokenized_data

# CountVectorizer를 사용하여 단어 사전 생성
countVec1 = CountVectorizer()
countVec1.fit(data['단지명'])


CountVectorizer()

In [11]:
def print_rmse_r2_score(model):
    # cv별 프린팅, 평균 저장
    model_name, rmse_list, r2_list = rmse_r2_cv(model)
    for i, (rmse, r2) in enumerate(zip(rmse_list, r2_list), start=1):
         print(f'{i} FOLDS: {model_name} RMSE: {rmse:.4f}, R2: {r2:.4f}')
    print(f'\n{model_name} mean RMSE: {np.mean(rmse_list):.4f}, mean R2: {np.mean(r2_list):.4f}')
    print('='*40)
    return model_name, np.mean(rmse_list), np.mean(r2_list)

def rmse_r2_cv(model):
    # cv별로 학습하는 함수
    tscv = TimeSeriesSplit(n_splits=10)
    rmse_list = []
    r2_list = []
    model_name = model.__class__.__name__
    for _, (train_index, test_index) in tqdm(enumerate(tscv.split(X), start=1), desc=f'{model_name} Cross Validations...', total=10):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf = model.fit(X_train, y_train)
        pred = clf.predict(X_test)
        rmse = RMSE(y_test, pred)
        r2 = r2_score(y_test, pred)
        rmse_list.append(rmse)
        r2_list.append(r2)
    return model_name, rmse_list, r2_list

def print_cv_rmse_r2_score(model, X, y):
    # CV를 통한 예측
    y_pred = cross_val_predict(model, X, y, cv=10)
    
    # RMSE 및 R2 계산
    rmse = mean_squared_error(y, y_pred, squared=False)
    r2 = r2_score(y, y_pred)
    
    model_name = model.__class__.__name__
    print(f'{model_name} RMSE: {rmse:.4f}, R2: {r2:.4f}')
    print('=' * 40)
    
    return model_name, rmse, r2

In [12]:

# 단어 사전을 사용하여 벡터화된 특성 생성
X_text = countVec1.transform(data['단지명'])
X_numeric = data[['전용면적(㎡)', '계약년월', '층', '건축나이', '거래건수', '기준금리', '법정동코드', '지하철역수']]
X = np.hstack((X_text.toarray(), X_numeric))

y = data['거래금액(만원)']



# 모델 리스트
models = [lrmodel, ridge, lasso, enet, d_tree, rf, model_xgb, model_lgb]

# Cross-validation으로 평가
results = []

for model in models:
    model_name, rmse, r2 = print_cv_rmse_r2_score(model, X, y)
    results.append({'Model': model_name, 'RMSE': rmse, 'R2': r2})

result_df = pd.DataFrame(results)
print(result_df)


LinearRegression RMSE: 276953.8320, R2: -491.2113


C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.59283e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.58679e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.53435e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.54168e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge RMSE: 5895.8019, R2: 0.7769


C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.771e+12, tolerance: 4.752e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.597e+12, tolerance: 5.069e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider in

KeyboardInterrupt: 

In [11]:
result_df

,Model,RMSE,R2
0,LinearRegression,6908.589336,0.693722
1,Ridge,6908.589328,0.693722
2,Lasso,6908.590029,0.693722
3,ElasticNet,6908.533340,0.693727
4,DecisionTreeRegressor,6573.862077,0.722682
5,RandomForestRegressor,5179.801678,0.827828
6,XGBRegressor,3693.792641,0.912445
7,LGBMRegressor,3803.011783,0.907191


In [1]:
X_predict = [[84, 202301,10,28 ,15,3.5,2915511500,0]]
X_predict_df= pd.DataFrame(X_predict,columns=['전용면적(㎡)', '계약년월','층', '건축나이','거래건수','기준금리','법정동코드','지하철역수'])

NameError: name 'pd' is not defined

In [12]:
print(f"LinearRegression: {lrmodel.predict(X_predict_df)[0]}")
print(f"Ridge: {ridge.predict(X_predict_df)[0]}")
print(f"Lasso: {lasso.predict(X_predict_df)[0]}")
print(f"ElasticNet: {enet.predict(X_predict_df)[0]}")
print(f"DecisionTreeRegressor: {d_tree.predict(X_predict_df)[0]}")
print(f"RandomForestRegressor: {rf.predict(X_predict_df)[0]}")
print(f"XGBRegressor: {model_xgb.predict(X_predict_df)[0]}")
print(f"LGBMRegressor :{model_lgb.predict(X_predict_df)[0]}")

NameError: name 'X_predict_df' is not defined